# Extension Project 
## Part 1: Extending the Analysis of AGN to include the X-ray band.
This script is intended to provide an extension of the work done in the first script. This extension in it's current form as no real aim and is intended to be exploratory. 


Notes for Project Extension 
* Finish off by fixing the Donley Selection
    * If time permits add the Stern Selection
* Maybe crossmatch with X-Ray data?
    * Focus on one field, CDFS with XMM-Newton data
    * chandra space 4Ms or 7Ms survey in topcat
    * Make liberal use of the crossmatching feature in TOPCAT or similarly follow on with this project and use the crossmatching feature in python
    * Use the XMM-Newton data to find the X-ray fluxes of the AGN
    * Use the seleciton criteria in Cowley et al. 2016 to select AGN by Luminosity and Hardness Ratio.
* Xray selection
* AGN Selection using luminosity threshold (CDFS)
* Possible UVJ selection
    * Potentially contaiminated by dusty torus
    * What we could do would be to first plot the UVJ diagram and see how the distribution looks in different redshift bins
    * Then we could plot the UVJ diagram for the AGN selected by the X-ray luminosity and see how that compares to the general population
    * we could also compare this to the IR selected AGNS, maybe using a subset of these AGN that are also X-ray selected.
* Could also look at how star-formation rate is effected between xray agn or ir agn?


HR <- Crossmatch the the HR from the XRAY ZFourge catalogue.



# Cross Matching ZFOURGE with XMM-Newton Data
* Focus on one field, CDFS with XMM-Newton data
* chandra space 4Ms survey in topcat (Check Paper)
    * Use the XMM-Newton data to find the X-ray fluxes of the AGN
* Make liberal use of the crossmatching feature in TOPCAT or similarly follow on with this project and create a cross-matching function in python
* Use the seleciton criteria in Cowley et al. 2016 to select AGN by Luminosity and Hardness Ratio.

In [1]:
# Begin by importing the required packages for the project
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import os

In [2]:
# Attempt to read in the fits file of the CDFS 7Ms survey or the Chandra 4Ms Survey.
# Noting for the 7Ms survey the the there is a flux but not a value for lumunosity.

# For now we will use the CDFS 4Ms survey (Xue et al. 2011)
# importing the fits file
file_path = os.path.join('crossmatch\CDFS_4Ms.fits')
xray_fits = fits.open(file_path)

xray_df = pd.DataFrame(np.array(xray_fits[1].data).byteswap().newbyteorder())  # Byteswap so that Pandas can read it


In [5]:
# The idea is now that we want to read in our data and use the xray selection technique outlined by in the AGN selection paper (Cowley et al. 2016)

0      0.46
1      0.43
2      1.59
3      1.89
4      1.61
       ... 
771    2.28
772    2.68
773    3.94
774    0.61
775    1.85
Name: BRat, Length: 776, dtype: float32